In [27]:
import requests
import smtplib
import copy
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from email.message import EmailMessage
from datetime import date

## Configurations

In [2]:
# Configure fake user agents
ua = UserAgent()
user_agent = ua.random
options = webdriver.ChromeOptions()
options.add_argument(f'--user-agent={user_agent}')

# Use headless browser
options.add_argument("--headless=new")


## Magic numbers for criteria

In [3]:
MARKET_CAP = 250
PE_TTM = 30
P_CF = 15
RELATIVE_PE = .7
ROA = .1
NET_PROFIT_MARGIN = .1
ROIC = .1
CAGR = .1
TOTAL_DEBT_EQUITY = 2
CURRENT_RATIO = 1
SHORT_FLOAT = .05
PAYOUT_RATIO = .2


## Scrape NASDAQ & NYSE listings from Finviz

In [4]:
companies = {}
FINVIZ_URLS = ['https://finviz.com/screener.ashx?v=111&f=cap_microover,exch_nasd,fa_netmargin_o10,fa_roa_o10,sh_short_low&ft=4&r=','https://finviz.com/screener.ashx?v=111&f=cap_microover,exch_nyse,fa_netmargin_o10,fa_roa_o10,sh_short_low&ft=4&r=']

# Start driver
driver = webdriver.Chrome(options=options)

# Scrape NASDAQ & NYSE based on initial criteria
for url in FINVIZ_URLS:
    
    last = False
    r = 1

    # Iterate through urls until last result page 
    while not last:
        
        current_url = url + str(r)
        driver.get(current_url)
        driver.set_window_size(1920, 1080)
            
        html = driver.page_source
        soup = BeautifulSoup(html)

        # Iterate through table
        table = soup.find('table', class_='styled-table-new is-rounded is-tabular-nums w-full screener_table')
        for row in table.find_all('tr'):

            # Scrape ticker symbols
            for element in row.find_all('a', class_='tab-link'):
                if element.text not in companies:
                    ticker = element.text
                    companies[ticker] = {'valid': True}
                else:
                    last = True

            # Scrape market caps
            for i, element in enumerate(row.find_all('a')):
                if i == 6 and not last:
                    companies[ticker]['market_cap'] = element.text

        r += 20

# Exclude companies under $ 250M
for c in companies:
    if companies[c]['market_cap'][-1] == 'M':
        if float(companies[c]['market_cap'][:-1]) < MARKET_CAP:
            companies[c]['valid'] = False

driver.quit()

## Scrape Yahoo Finance

In [5]:
YAHOO_URL_ONE = 'https://finance.yahoo.com/quote/'
YAHOO_URL_TWO = '/key-statistics'

driver = webdriver.Chrome(options=options)
driver.set_window_size(1920, 1080)
#companies_test = {}
#companies_test['AAPL'] = {'valid': True, 'market_cap': '2.91T'}
#companies_test['META'] = {'valid': True, 'market_cap': '1.20T'}

for i, company in enumerate(companies):
    if companies[company]['valid']:
        ua = UserAgent()
        user_agent = ua.random
        headers = {"User-Agent": user_agent}
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent":user_agent})

        # Yahoo summary page
        summary_url = YAHOO_URL_ONE + company
        page = requests.get(summary_url, headers=headers)
        soup = BeautifulSoup(page.content)
        pe = soup.find('fin-streamer',attrs={"data-field": "trailingPE"})
        pe = pe.text if pe else pe
        try:
            pe = float(pe)
            companies[company]['valid'] = False if pe >= PE_TTM else True
        except:
            pass
        companies[company]['pe'] = pe
        companies[company]['url'] = summary_url

        # Yahoo statistics page
        if companies[company]['valid']:
            
            stats_url = summary_url + YAHOO_URL_TWO
            driver.get(stats_url)
            
            operating_margin = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]/div/section[2]/table/tbody/tr[2]/td[2]')
            operating_margin = operating_margin.text[:-1] if operating_margin else operating_margin
            try:
                operating_margin = float(operating_margin) / 100
            except:
                pass
            companies[company]['operating_margin'] = operating_margin
            #companies_test[company]['valid'] = False if operating_margin

            total_debt_equity = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]/div/section[5]/table/tbody/tr[4]/td[2]')
            total_debt_equity = total_debt_equity.text[:-1] if total_debt_equity else total_debt_equity
            try:
                total_debt_equity = float(total_debt_equity) / 100
                companies[company]['valid'] = False if total_debt_equity >= TOTAL_DEBT_EQUITY else True
            except:
                pass
            companies[company]['total_debt_equity'] = total_debt_equity
            

            current_ratio = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]/div/section[5]/table/tbody/tr[5]/td[2]')
            current_ratio = current_ratio.text if current_ratio else current_ratio
            try:
                current_ratio = float(current_ratio)
                companies[company]['valid'] = False if current_ratio <= CURRENT_RATIO else True
            except:
                pass
            companies[company]['current_ratio'] = current_ratio
            
            
            short_float = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[2]/div/section[2]/table/tbody/tr[10]/td[2]')
            short_float = short_float.text[:-1] if short_float else short_float
            try:
                short_float = float(short_float) / 100
                companies[company]['valid'] = False if short_float >= SHORT_FLOAT else True
            except:
                pass
            companies[company]['short_float'] = short_float
            

            payout_ratio = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/article/div/section[2]/div/section[3]/table/tbody/tr[6]/td[2]')
            payout_ratio = payout_ratio.text[:-1] if payout_ratio else payout_ratio
            try:
                payout_ratio = float(payout_ratio) / 100
                companies[company]['valid'] = False if payout_ratio >= PAYOUT_RATIO else True
            except:
                pass
            companies[company]['payout_ratio'] = payout_ratio
        

    # Fullratio page
    #if companies_test[company]['valid']:
    print(f'{i+1} of {len(companies)} screened')
        

driver.quit()

1 of 317 screened
2 of 317 screened
3 of 317 screened
4 of 317 screened
5 of 317 screened
6 of 317 screened
7 of 317 screened
8 of 317 screened
9 of 317 screened
10 of 317 screened
11 of 317 screened
12 of 317 screened
13 of 317 screened
14 of 317 screened
15 of 317 screened
16 of 317 screened
17 of 317 screened
18 of 317 screened
19 of 317 screened
20 of 317 screened
21 of 317 screened
22 of 317 screened
23 of 317 screened
24 of 317 screened
25 of 317 screened
26 of 317 screened
27 of 317 screened
28 of 317 screened
29 of 317 screened
30 of 317 screened
31 of 317 screened
32 of 317 screened
33 of 317 screened
34 of 317 screened
35 of 317 screened
36 of 317 screened
37 of 317 screened
38 of 317 screened
39 of 317 screened
40 of 317 screened
41 of 317 screened
42 of 317 screened
43 of 317 screened
44 of 317 screened
45 of 317 screened
46 of 317 screened
47 of 317 screened
48 of 317 screened
49 of 317 screened
50 of 317 screened
51 of 317 screened
52 of 317 screened
53 of 317 screened
54

## Clean dict & convert to excel file

In [29]:
company_copy = copy.deepcopy(companies)
valid_output = {} 

for c in companies:
    
    # Add valid companies to new dict
    if companies[c]['valid']:
        valid_output[c] = company_copy[c]

        # Convert floats to formatted strings
        # for key in valid_output[c]:
        #     if key in ['operating_margin', 'total_debt_equity', 'short_float', 'payout_ratio']:
        #         if valid_output[c][key] not in ['-', '--']:
        #             valid_output[c][key] = str(round(valid_output[c][key] * 100, 2)) + '%'

df = pd.DataFrame(data=valid_output).transpose()
excel_output = f'{date.today()} Stock Screener.xlsx'
df.to_excel(excel_output)

## Send email

In [30]:
# Create a secure SSL context
PW = 'umtw vgqo wgdn egsg'
sender_email = 'liamstockscreener1@gmail.com'
receiver_email = 'mags.liam@gmail.com'

msg = EmailMessage()
msg['Subject'] = f'Stock Screener: {date.today()}'
msg['From'] = 'Leeham'
msg['To'] = receiver_email

with open(excel_output, 'rb') as f:
        file_data = f.read()
msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename=excel_output)

MyServer = smtplib.SMTP('smtp.gmail.com', 587)
MyServer.starttls()
MyServer.login(sender_email, PW)
MyServer.send_message(msg)
MyServer.quit()



(221,
 b'2.0.0 closing connection 6a1803df08f44-6ae4a73e141sm9450166d6.4 - gsmtp')